In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

# Quantum Encryption Game

This notebook is meant to facilitate a "game" to demonstrate the concept behind quantum encryption.

## Background

Cryptography is typically done by the sender scrambling a communication in some way and the receiver knowing how to unscramble it. Let's say Alice wants Bob to send her his super secret chili recipe. Alice will make up some scheme for scrambling/unscrambling the data. She will send Bob instructions for doing the scrambling while only she knows how to unscramble it. The asymmetry here is very important -- even though Bob knows how to scramble the data, he (nor anyone else but Alice) should know how to unscramble it. The scrambling instructions are known as a *public key* and Alice's secret unscrambling instructions are the *private key*. The overall strategy I've described here is called *public key cryptography*.

RSA is a public key cryptography algorithm commonly used for encryption over the internet. Under RSA, the public key is a product of two prime numbers, $d=p\cdot q$. Then, when Bob goes to encrypt his message, $x$, (which is really a number because that's how computers think of data), he will perform a modular operation $y=x^e\%d$. He will then send his encrypted number, $y$, to Alice. Alice knows the prime numbers that went into the public key, and she is able to calculate a "magic number," $m$ which will undo the operation that Bob performed, $x=y^m\%d$.
I certainly have not proven that this works, nor have I explained how to find $m$. [This computerphile video](https://www.youtube.com/watch?v=JD72Ry60eP4) goes into much more detail. But the main point is that anyone can know the public key, $d$, but only Alice, who knows the prime number $p$ and $q$ can decrypt the message.

You might be noticing the vulnerability here. If someone were to figure out $p$ and $q$, they can intercept the message and learn Bob's secret chili recipe. The solution here, so far, is to use very very large numbers (on order 300 digits per prime number). In fact, if you watch the video above, you can learn how to actually see the numbers your computer is using for encryption. But as computers get faster, they can factor larger numbers faster. So we use even larger numbers. It's a bit of a race. It becomes an even more significant problem when you realize a bad actor doesn't need to decrypt your message *now*. In principle, they can store public keys and messages for years until computer power catches up and decrypt later. Depending on the nature of the information, this could be a real issue.

Enter **quantum computers**. In a bit of irony, quantum computers will accelerate the problem and provide a very robust solution. First the accelerated problem. Shor's algorithm was developed in 1994, which shows that quantum computers (computers that "think" using non-binary "qubits" rather than the traditional 1's and 0's) will be able to factor very large numbers much faster. With quantum computers the problem reduces to polynomial time, rather than sub-exponential time for classical computing. So once quantum computing technology has developed to the point that they can hold and manipulate many qubits, RSA will essentially be obsolete.

But **quantum cryptography** offers a method to send a secret key with a guarantee that no one saw it in transmission. This is certainly one of those quantum mechanical ideas that makes you say "huh? how's that possible?" Let's dig into it.

### Quantum Cryptography Basics

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\up}{\ket{\uparrow}}
\newcommand{\down}{\ket{\downarrow}}$$
Alice and Bob need to agree on a secret key. Alice will prepare a bunch of quantum states using two non-commuting observables. For this example, let's say she uses the spin of electrons along the $z-$ and $x-$axes, so our observables are $S_z$ and $S_x$. So for each electron she will know the spin state, $\up$ or $\down$, along a given axis, $z$ or $x$. So let's make up a sequence:

$\text{Alice states} = \down_x\; \down_z\; \up_x\; \down_z\; \up_z\; \down_z\; \up_x\; \down_z\; \up_x\; \up_x$

Alice knows these to be the states of the electrons, but she does not share it with anyone! Instead she passes the actual electrons to Bob. Bob then picks a sequence of $x$ and $z$ to measure the electrons. In the cases where Bob happens to pick the same axis as Alice, they will agree. If Bob picks the opposing axis, it's a 50-50 whether they will agree with the spin or not. Let's try it.

Bob selects the sequence: $x\;x\;z\;x\;x\;z\;z\;x\;x\;z$ (I'm using random numbers to generate these, by the way.)

Bob will measure the following state.

$\text{Bob states} = $<font color='green'>$\down_x\;$</font><font color='red'>$\down_x\; \up_z\; \down_x\; \down_x\;$</font><font color='green'>$\down_z\;$</font><font color='red'> $\down_z\; \down_x\;$</font><font color='green'>$ \up_x\;$</font><font color='red'>$ \up_z$</font>

I've colored the states green where Alice and Bob chose the same axis, and red otherwise. The next step is they call each other up and compare the *axes* that they measured - **NOT** the results. But once they know that they agree with those green states above, they have a sequence ($\down\;\down\;\up$) that only they know. This becomes the secret key. This demo was using only a small number of states, but with sufficient numbers they can create a sequence to encrypt and decrypt the data that only they know.</font>
    
How do we know that no one intercepted the key? Well, if Steven the spy were to measure the states in between, he will alter them. Any instance where Steven uses a different axis than Alice did, he will collapse the electrons wavefunction to the wrong axis, which means even if Bob measures the same axis as Alice, he has a 50-50 chance of getting the wrong state. They won't actually know that something went wrong until Bob tries to send a test message, but when it arrives scrambled by the wrong key, they will know their key is broken.

## The Game

Ok, let's try this out using large numbers of states. In the notebook I will pick random numbers, but in the classroom the students can pick their own state and walk through the process.

In [2]:
nstates = 100
Alice_axes = np.random.choice(('z', 'x'), nstates)  # <= "Alice" students choose this!
Alice_spins = np.random.choice(('|↓⟩', '|↑⟩'), nstates) # <= instructor determines this, shares only with Alice
Alice_states = [s+a for (s,a) in zip(Alice_spins, Alice_axes)] # convenience for printing

Bob_axes = np.random.choice(('z', 'x'), nstates)  # <= "Bob" students choose this!
Bob_spins = [alice_s if alice_a == bob_a else np.random.choice(('|↓⟩', '|↑⟩'))
             for (alice_a, alice_s, bob_a) in zip(Alice_axes, Alice_spins, Bob_axes)]
Bob_states = [s+a for (s,a) in zip(Bob_spins, Bob_axes)] # convenience for printing

At this point Alice and Bob each have their sequence of ups and down and they know which axis they used.
Although you would not do this in reality, they may want to compare their lists of state (ups and downs) at this stage - they should see they agree with each other about 75% of the time.

In [3]:
match = np.mean([1 if a == b else 0 for (a, b) in zip(Alice_spins, Bob_spins)])
print(f'Alice and Bob spins match {round(100 * match, 1)}%.')

for a, b in zip(Alice_states, Bob_states):
    print(a + '\t' + b)

Alice and Bob spins match 78.0%.
|↑⟩z	|↑⟩z
|↑⟩x	|↑⟩x
|↑⟩z	|↑⟩z
|↓⟩z	|↓⟩z
|↑⟩x	|↑⟩z
|↓⟩z	|↓⟩z
|↑⟩z	|↑⟩x
|↑⟩z	|↑⟩z
|↓⟩x	|↓⟩x
|↓⟩x	|↑⟩z
|↓⟩x	|↓⟩z
|↓⟩x	|↓⟩x
|↑⟩x	|↑⟩z
|↓⟩z	|↓⟩x
|↓⟩z	|↓⟩x
|↑⟩x	|↑⟩x
|↑⟩x	|↑⟩z
|↓⟩z	|↓⟩x
|↑⟩z	|↑⟩x
|↓⟩z	|↓⟩x
|↓⟩x	|↓⟩z
|↓⟩x	|↓⟩x
|↓⟩z	|↓⟩z
|↓⟩x	|↑⟩z
|↑⟩z	|↑⟩x
|↑⟩x	|↓⟩z
|↓⟩x	|↓⟩z
|↓⟩z	|↓⟩z
|↑⟩z	|↑⟩x
|↓⟩z	|↓⟩x
|↓⟩x	|↓⟩z
|↓⟩x	|↓⟩x
|↑⟩x	|↑⟩z
|↑⟩z	|↑⟩x
|↓⟩z	|↑⟩x
|↓⟩x	|↑⟩z
|↓⟩z	|↓⟩x
|↓⟩x	|↓⟩x
|↑⟩z	|↓⟩x
|↓⟩x	|↓⟩x
|↓⟩x	|↑⟩z
|↑⟩x	|↑⟩x
|↓⟩z	|↓⟩z
|↓⟩z	|↓⟩x
|↓⟩x	|↑⟩z
|↓⟩z	|↓⟩z
|↑⟩x	|↑⟩z
|↓⟩x	|↓⟩x
|↑⟩x	|↓⟩z
|↑⟩z	|↑⟩z
|↑⟩z	|↓⟩x
|↓⟩z	|↑⟩x
|↑⟩z	|↑⟩z
|↓⟩z	|↑⟩x
|↑⟩z	|↑⟩z
|↑⟩z	|↓⟩x
|↓⟩z	|↓⟩x
|↑⟩z	|↑⟩z
|↑⟩z	|↑⟩z
|↑⟩z	|↑⟩x
|↓⟩x	|↓⟩z
|↑⟩x	|↑⟩x
|↑⟩z	|↓⟩x
|↓⟩z	|↓⟩z
|↑⟩z	|↑⟩z
|↓⟩x	|↓⟩x
|↓⟩x	|↓⟩z
|↑⟩x	|↑⟩z
|↑⟩z	|↑⟩z
|↑⟩x	|↓⟩z
|↓⟩x	|↓⟩x
|↑⟩z	|↑⟩z
|↑⟩z	|↑⟩x
|↑⟩x	|↑⟩x
|↓⟩z	|↓⟩x
|↓⟩z	|↓⟩z
|↓⟩x	|↓⟩x
|↓⟩z	|↑⟩x
|↓⟩z	|↓⟩z
|↓⟩x	|↑⟩z
|↓⟩x	|↓⟩z
|↓⟩x	|↑⟩z
|↓⟩z	|↑⟩x
|↓⟩x	|↓⟩x
|↓⟩x	|↓⟩z
|↑⟩z	|↓⟩x
|↑⟩z	|↑⟩z
|↑⟩x	|↑⟩x
|↓⟩z	|↓⟩z
|↑⟩x	|↑⟩x
|↓⟩z	|↓⟩z
|↓⟩z	|↓⟩z
|↑⟩z	|↑⟩z
|↓⟩z	|↓⟩x
|↓⟩x	|↓⟩z
|↓⟩x	|↑⟩z
|↓⟩z	|↓

Let's find which states they agreed on the axis, and finally come up with our secret key.

In [4]:
matched_axes = [1 if a == b else 0 for (a, b) in zip(Alice_axes, Bob_axes)]
Alice_key = np.array(Alice_spins)[np.where(matched_axes)[0]]
Bob_key = np.array(Bob_spins)[np.where(matched_axes)[0]]
# Just to show they do in fact agree
for a, b in zip(Alice_key, Bob_key):
    print(a + '\t' + b)
print(np.all(Alice_key == Bob_key))

|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
True


## Steven the Spy

Now let's introduce Steven the Spy. After Alice set up her data, Steven will intercept it. He will produce his own set up spins and axes and pass those states along to Bob who will make his measurements as if he received directly from Alice.

In [5]:
Steven_axes = np.random.choice(('z', 'x'), nstates)  # <= "Steven" students choose this!
Steven_spins = [alice_s if alice_a == steven_a else np.random.choice(('|↓⟩', '|↑⟩'))
             for (alice_a, alice_s, steven_a) in zip(Alice_axes, Alice_spins, Steven_axes)]
Steven_states = [s+a for (s,a) in zip(Steven_spins, Steven_axes)] # convenience for printing

Bob_axes = np.random.choice(('z', 'x'), nstates)  # <= "Bob" students choose this!
Bob_spins = [steven_s if steven_a == bob_a else np.random.choice(('|↓⟩', '|↑⟩'))
             for (steven_a, steven_s, bob_a) in zip(Steven_axes, Steven_spins, Bob_axes)]
Bob_states = [s+a for (s,a) in zip(Bob_spins, Bob_axes)] # convenience for printing

In [6]:
match = np.mean([1 if a == b else 0 for (a, b) in zip(Alice_spins, Bob_spins)])
print(f'Alice and Bob spins match {round(100 * match, 1)}%.')

for a, b in zip(Alice_states, Bob_states):
    print(a + '\t' + b)

Alice and Bob spins match 66.0%.
|↑⟩z	|↑⟩z
|↑⟩x	|↓⟩z
|↑⟩z	|↓⟩x
|↓⟩z	|↓⟩z
|↑⟩x	|↓⟩z
|↓⟩z	|↑⟩x
|↑⟩z	|↑⟩z
|↑⟩z	|↓⟩x
|↓⟩x	|↓⟩z
|↓⟩x	|↓⟩x
|↓⟩x	|↑⟩x
|↓⟩x	|↓⟩x
|↑⟩x	|↑⟩z
|↓⟩z	|↑⟩z
|↓⟩z	|↓⟩x
|↑⟩x	|↓⟩x
|↑⟩x	|↑⟩x
|↓⟩z	|↓⟩x
|↑⟩z	|↑⟩x
|↓⟩z	|↓⟩x
|↓⟩x	|↓⟩x
|↓⟩x	|↓⟩x
|↓⟩z	|↑⟩x
|↓⟩x	|↓⟩x
|↑⟩z	|↑⟩z
|↑⟩x	|↓⟩z
|↓⟩x	|↓⟩x
|↓⟩z	|↓⟩x
|↑⟩z	|↑⟩z
|↓⟩z	|↑⟩z
|↓⟩x	|↓⟩x
|↓⟩x	|↓⟩z
|↑⟩x	|↑⟩x
|↑⟩z	|↑⟩z
|↓⟩z	|↓⟩x
|↓⟩x	|↑⟩x
|↓⟩z	|↓⟩z
|↓⟩x	|↓⟩z
|↑⟩z	|↓⟩x
|↓⟩x	|↓⟩x
|↓⟩x	|↑⟩x
|↑⟩x	|↑⟩x
|↓⟩z	|↓⟩x
|↓⟩z	|↓⟩z
|↓⟩x	|↓⟩x
|↓⟩z	|↓⟩z
|↑⟩x	|↓⟩z
|↓⟩x	|↓⟩x
|↑⟩x	|↓⟩z
|↑⟩z	|↓⟩x
|↑⟩z	|↑⟩z
|↓⟩z	|↓⟩x
|↑⟩z	|↓⟩z
|↓⟩z	|↓⟩z
|↑⟩z	|↑⟩z
|↑⟩z	|↑⟩x
|↓⟩z	|↑⟩z
|↑⟩z	|↓⟩z
|↑⟩z	|↑⟩z
|↑⟩z	|↑⟩z
|↓⟩x	|↑⟩z
|↑⟩x	|↓⟩z
|↑⟩z	|↑⟩z
|↓⟩z	|↓⟩z
|↑⟩z	|↓⟩z
|↓⟩x	|↓⟩z
|↓⟩x	|↓⟩x
|↑⟩x	|↓⟩z
|↑⟩z	|↑⟩z
|↑⟩x	|↓⟩x
|↓⟩x	|↓⟩x
|↑⟩z	|↑⟩z
|↑⟩z	|↓⟩x
|↑⟩x	|↓⟩x
|↓⟩z	|↑⟩z
|↓⟩z	|↓⟩z
|↓⟩x	|↑⟩z
|↓⟩z	|↓⟩x
|↓⟩z	|↓⟩z
|↓⟩x	|↓⟩x
|↓⟩x	|↓⟩z
|↓⟩x	|↓⟩z
|↓⟩z	|↓⟩x
|↓⟩x	|↓⟩x
|↓⟩x	|↓⟩z
|↑⟩z	|↑⟩z
|↑⟩z	|↑⟩z
|↑⟩x	|↑⟩z
|↓⟩z	|↑⟩x
|↑⟩x	|↓⟩z
|↓⟩z	|↓⟩z
|↓⟩z	|↑⟩x
|↑⟩z	|↑⟩z
|↓⟩z	|↓⟩z
|↓⟩x	|↓⟩x
|↓⟩x	|↑⟩x
|↓⟩z	|↓

In [7]:
matched_axes = [1 if a == b else 0 for (a, b) in zip(Alice_axes, Bob_axes)]
Alice_key = np.array(Alice_spins)[np.where(matched_axes)[0]]
Bob_key = np.array(Bob_spins)[np.where(matched_axes)[0]]
# Just to show they do in fact agree
for a, b in zip(Alice_key, Bob_key):
    print(a + '\t' + b)
print(np.all(Alice_key == Bob_key))
print(f'Now the keys only match {round(100 * np.mean(Alice_key == Bob_key), 1)}%')

|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↑⟩
|↑⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↓⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↑⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↑⟩
|↑⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↓⟩
|↓⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↑⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↓⟩
|↓⟩	|↑⟩
|↓⟩	|↓⟩
|↓⟩	|↑⟩
False
Now the keys only match 74.6%
